In [1]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

import bb_rt_helper

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [2]:
import yaml

In [3]:
with open('refined_corridors_hotspots.yaml', 'r') as f:
    all_corr_hs = yaml.safe_load(f)

In [4]:
## future parameter
district_param = '03 - Marysville'

In [5]:
all_corr_hs

{'districts': [{'district': '03 - Marysville',
   'date_override': '2022-02-08',
   'get_sorted_corr': False,
   'get_sorted_hs': False,
   'corr_extra_fm': None,
   'hs_extra_fm': None,
   'corridors': [{'corridor': {'itp_id': 273,
      'route_filter': None,
      'parts': [{'part': None, 'shape_id': '40769', 'stop_bounds': [3, 39]},
       {'part': None, 'shape_id': '40768', 'stop_bounds': [19, 34]}]}},
    {'corridor': {'itp_id': 273,
      'route_filter': None,
      'parts': [{'part': None, 'shape_id': '40783', 'stop_bounds': [28, 58]},
       {'part': None, 'shape_id': '40784', 'stop_bounds': [11, 19]}]}},
    {'corridor': {'itp_id': 273,
      'route_filter': None,
      'parts': [{'part': None,
        'shape_id': '40744',
        'stop_bounds': [22, 36]}]}},
    {'corridor': {'itp_id': 273,
      'route_filter': None,
      'parts': [{'part': None,
        'shape_id': '40735',
        'stop_bounds': [7, 34]}]}}],
   'hotspots': [{'hotspot': {'itp_id': 273,
      'route_filter

In [6]:
# find district information matching parameter
for district in all_corr_hs['districts']:
    if district['district'] == district_param:
        district_dict = district

In [7]:
district_dict

{'district': '03 - Marysville',
 'date_override': '2022-02-08',
 'get_sorted_corr': False,
 'get_sorted_hs': False,
 'corr_extra_fm': None,
 'hs_extra_fm': None,
 'corridors': [{'corridor': {'itp_id': 273,
    'route_filter': None,
    'parts': [{'part': None, 'shape_id': '40769', 'stop_bounds': [3, 39]},
     {'part': None, 'shape_id': '40768', 'stop_bounds': [19, 34]}]}},
  {'corridor': {'itp_id': 273,
    'route_filter': None,
    'parts': [{'part': None, 'shape_id': '40783', 'stop_bounds': [28, 58]},
     {'part': None, 'shape_id': '40784', 'stop_bounds': [11, 19]}]}},
  {'corridor': {'itp_id': 273,
    'route_filter': None,
    'parts': [{'part': None, 'shape_id': '40744', 'stop_bounds': [22, 36]}]}},
  {'corridor': {'itp_id': 273,
    'route_filter': None,
    'parts': [{'part': None, 'shape_id': '40735', 'stop_bounds': [7, 34]}]}}],
 'hotspots': [{'hotspot': {'itp_id': 273,
    'route_filter': None,
    'parts': [{'part': None, 'shape_id': '40735', 'stop_bounds': [22, 25]}]}},
 

In [8]:
fm_dict = {}
corridors = []
hotspots = []
# override default date if needed (so far only d3/SacRT)
analysis_date = dt.date.fromisoformat(district_dict['date_override']) if district_dict['date_override'] else dt.date(2022, 5, 4)

In [9]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist=district_param, category='corridor',
                                       get_sorted = district_dict['get_sorted_corr'],
                                       analysis_date=analysis_date, fm_dict = fm_dict)

03 - Marysville corridor empty with default util, retrying with get_sorted


loading filter/mapper for 48
no filtermapper for 48
loading filter/mapper for 251
no filtermapper for 251
loading filter/mapper for 273
found parquet
{273: <rt_analysis.rt_filter_map_plot.RtFilterMapper object at 0x7fbb44652ac0>}


In [10]:
# save speedmap views across all operators for future combined map
corr_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    corr_speed_views += [fm_dict[key].detailed_map_view.copy()]

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [11]:
district_dict['corridors'][0]['corridor']

{'itp_id': 273,
 'route_filter': None,
 'parts': [{'part': None, 'shape_id': '40769', 'stop_bounds': [3, 39]},
  {'part': None, 'shape_id': '40768', 'stop_bounds': [19, 34]}]}

In [12]:
def parse_corridor(corridor_dict):
    '''
    Construct analysis corridor and calculate corridor metrics for each corridor
    specified. Also, add corridor information to overall list for future map/export
    
    corridor_dict: dict, item['corridor'] for any item in district_dict['corridors']
    '''
    global corridors
    subcorridors = []
    for part in corridor_dict['parts']:
        fm_dict[corridor_dict['itp_id']].autocorridor(part['shape_id'], part['stop_bounds'])
        _corridor = fm_dict[corridor_dict['itp_id']].corridor.copy()
        subcorridors += [_corridor]
    full_corridor = pd.concat(subcorridors).dissolve()
    fm_dict[corridor_dict['itp_id']].add_corridor(full_corridor)
    if corridor_dict['route_filter']:
        fm_dict[corridor_dict['itp_id']].set_filter(route_names = corridor_dict['route_filter'])
    else:
        fm_dict[corridor_dict['itp_id']].reset_filter()
    fm_dict[corridor_dict['itp_id']].corridor_metrics()
    display(fm_dict[corridor_dict['itp_id']].quick_map_corridor())
    measured_corridor = fm_dict[corridor_dict['itp_id']].corridor.copy()
    corridors += [measured_corridor]
    return

In [13]:
# parse_corridor(district_dict['corridors'][0]['corridor'])

In [14]:
for corridor in district_dict['corridors']:
    parse_corridor(corridor['corridor'])

In [15]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist=district_param, category='hotspot',
                                       get_sorted = district_dict['get_sorted_hs'],
                                       analysis_date=analysis_date, fm_dict = fm_dict)

03 - Marysville hotspot empty with default util, retrying with get_sorted


loading filter/mapper for 48
no filtermapper for 48
loading filter/mapper for 251
no filtermapper for 251
loading filter/mapper for 273
{273: <rt_analysis.rt_filter_map_plot.RtFilterMapper object at 0x7fbb44652ac0>}


In [16]:
hs_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    hs_speed_views += [fm_dict[key].detailed_map_view.copy()]

/opt/conda/lib/python3.9/site-packages/shapely/ops.py:639: ShapelyDeprecationWarning: GeometryTypeError will derive from ShapelyError and not TypeError or ValueError in Shapely 2.0.
/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/rt_delay/rt_analysis/rt_filter_map_plot.py:395: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.



In [17]:
def parse_hotspot(hotspot_dict):
    '''
    Construct analysis corridor and calculate corridor metrics for each hotspot
    specified. Also, add hotspot information to overall list for future map/export
    
    hotspot_dict: dict, item['hotspot'] for any item in district_dict['hotspots']
    '''
    global hotspots
    subcorridors = []
    for part in hotspot_dict['parts']:
        fm_dict[hotspot_dict['itp_id']].autocorridor(part['shape_id'], part['stop_bounds'])
        _corridor = fm_dict[hotspot_dict['itp_id']].corridor.copy()
        subcorridors += [_corridor]
    full_corridor = pd.concat(subcorridors).dissolve()
    fm_dict[hotspot_dict['itp_id']].add_corridor(full_corridor)
    if hotspot_dict['route_filter']:
        fm_dict[hotspot_dict['itp_id']].set_filter(route_names = hotspot_dict['route_filter'])
    else:
        fm_dict[hotspot_dict['itp_id']].reset_filter()
    fm_dict[hotspot_dict['itp_id']].corridor_metrics()
    display(fm_dict[hotspot_dict['itp_id']].quick_map_corridor())
    measured_corridor = fm_dict[hotspot_dict['itp_id']].corridor.copy()
    hotspots += [measured_corridor]
    return

In [18]:
for hotspot in district_dict['hotspots']:
    parse_hotspot(hotspot['hotspot'])

In [19]:
# save speedmap views across all operators for future combined map
all_speed_segs = pd.concat(hs_speed_views)
all_corr_segs = pd.concat(corr_speed_views)
# full join to preserve all unique segments, since they can differ
# from hotspot to corridor (or indeed across operators)
all_segs = all_speed_segs >> full_join(_, all_corr_segs)

In [20]:
bb_rt_helper.bb_map_all(hotspots, corridors, all_segs, district = district_param)

/home/jovyan/data-analyses/one_hundred_recs/bb_rt_helper.py:77: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

/home/jovyan/data-analyses/one_hundred_recs/bb_rt_helper.py:77: UserWarning: Geometry is in a geographic CRS. Results from 'centroid' are likely incorrect. Use 'GeoSeries.to_crs()' to re-project geometries to a projected CRS before this operation.

